In [103]:
import os

from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_validator import ExcelValidator
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_finder import TableFinder
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from ast_transformation.formula_generator import FormulaGenerator
from ast_transformation.formula_checker import FormulaChecker

In [104]:
current_directory = os.getcwd()

parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
data_directory = os.path.join(parent_directory, 'data')

project_name = 'test_excel_7'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")

In [105]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)

In [106]:
is_valid = ExcelValidator.validate_excel(excel_reduced)

In [107]:
if not is_valid:
    raise Exception("Excel file is not valid")

In [108]:
excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)

In [109]:
extracted_tables, data = TableFinder.find_tables(excel_reduced_clean)

In [110]:
series_data = SeriesExtractor.extract_table_details(extracted_tables, data)

In [111]:
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)

In [112]:
if not is_compatible:
    raise Exception("Excel file is not compatible")

In [113]:
series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, data=data)

In [114]:
series_dict

{'Sheet1': [Series(series_id=SeriesId(sheet_name='Sheet1', series_header='App', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='App', formulas=[None, None], values=['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>),
  Series(series_id=SeriesId(sheet_name='Sheet1', series_header='Category', series_header_cell_row=1, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='Category', formulas=[None, None], values=['ART_AND_DESIGN', 'ART_AND_DESIGN'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=2, coordinate='B2', value=None, value_type=None), series_length

In [115]:
series_mapping = SeriesMapper.map_series(series_dict)

In [116]:
series_mapping

{Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None): {Cell(column=1, row=2, coordinate=None, value=None, value_type=None): (0,
   Series(series_id=SeriesId(sheet_name='Sheet1', series_header='App', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='App', formulas=[None, None], values=['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana'], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.STR: 'str'>)),
  Cell(column=1, row=3, coordinate=None, value=None, value_type=None): (1,
   Series(series_id=SeriesId(sheet_name='Sheet1', series_header='App', series_header_cell_row=1, series_header_cell_column=1), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='App', formulas=[

In [117]:
series_iterator = SeriesIterator.iterate_series(series_dict)

In [118]:
series_list = [series for series in series_iterator if series.formulas != [None, None]]

Series Id can be concatenation of sheet_name, header_name, header_column_index, header_row_index

In [119]:
for series in series_list:
    formula_1 = series.formulas[0]
    formula_1_ast = FormulaParser.parse_formula(formula_1)
    series_implementer = SeriesImplementer(series_mapping, sheet_name = series.worksheet.sheet_name)
    formula_1_ast_series = series_implementer.update_ast(formula_1_ast)

    formula_2 = series.formulas[1]
    formula_2_ast = FormulaParser.parse_formula(formula_2)
    formula_2_ast_series = series_implementer.update_ast(formula_2_ast)
    
    sheet_name = series.worksheet.sheet_name

    series_list = series_dict.get(sheet_name)

    ast_generator = FormulaGenerator.get_ast_generator(formula_1_ast_series, formula_2_ast_series, series_list)

    formulas_are_correct, formula_1_ast_new, formula_2_ast_new = FormulaChecker.check_formulas(ast_generator)
    if not formulas_are_correct:
        print("formula_2_ast_series")
        print(formula_2_ast_series)
        print("formula_2_ast_new")
        print(formula_2_ast_new)
        raise Exception("Formulas are not correct")